first download sample dataset from Disney-VideoGeneration-Dataset and store it in /data/zihao/Disney-VideoGeneration-Dataset

then use the first frame of videos as images to test i2v

In [ ]:
# do not need to run again
import os
import cv2

dataset_path = "/data/zihao/Disney-VideoGeneration-Dataset"
videos_txt_path = os.path.join(dataset_path, "videos.txt")
images_dir = os.path.join(dataset_path, "images")
images_txt_path = os.path.join(dataset_path, "images.txt")

# make images folder
os.makedirs(images_dir, exist_ok=True)

# 读取 videos.txt
with open(videos_txt_path, "r") as f:
    video_paths = [line.strip() for line in f.readlines()]

image_paths = []

for video_path in video_paths:
    video_full_path = os.path.join(dataset_path, video_path)
    
    cap = cv2.VideoCapture(video_full_path)
    success, frame = cap.read()
    cap.release()
    
    if success:
        image_name = os.path.splitext(os.path.basename(video_path))[0] + ".jpg"
        image_path = os.path.join(images_dir, image_name)
        cv2.imwrite(image_path, frame)
        image_paths.append(os.path.relpath(image_path, dataset_path))
    else:
        print(f"failed in {video_path}")


with open(images_txt_path, "w") as f:
    for image_path in image_paths:
        f.write(image_path + "\n")

setup model configurations (same as the train_ddp_i2v.sh)

In [1]:
import os
import argparse
%load_ext autoreload
from finetune.schemas import Args
def get_training_args():
    parser = argparse.ArgumentParser(description="Training arguments for CogVideoX-5B-I2V")
    
    # Model Configuration
    parser.add_argument("--model_path", type=str, default="THUDM/CogVideoX-5B-I2V")
    parser.add_argument("--model_name", type=str, default="cogvideox-i2v")
    parser.add_argument("--model_type", type=str, default="i2v")
    parser.add_argument("--training_type", type=str, default="lora")
    
    # Output Configuration
    parser.add_argument("--output_dir", type=str, default="finetune_output")
    parser.add_argument("--report_to", type=str, default="tensorboard")
    
    # Data Configuration
    parser.add_argument("--data_root", type=str, default="/data/zihao/Disney-VideoGeneration-Dataset")
    parser.add_argument("--caption_column", type=str, default="prompt.txt")
    parser.add_argument("--video_column", type=str, default="videos.txt")
    parser.add_argument("--image_column", type=str, default="images.txt")  # Empty by default
    parser.add_argument("--train_resolution", type=str, default="49x480x720")
    
    # Training Configuration
    parser.add_argument("--train_epochs", type=int, default=10)
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--batch_size", type=int, default=1)
    parser.add_argument("--gradient_accumulation_steps", type=int, default=1)
    parser.add_argument("--mixed_precision", type=str, default="bf16")
    
    # System Configuration
    parser.add_argument("--num_workers", type=int, default=8)
    parser.add_argument("--pin_memory", type=bool, default=True)
    parser.add_argument("--nccl_timeout", type=int, default=1800)
    
    # Checkpointing Configuration
    parser.add_argument("--checkpointing_steps", type=int, default=10)
    parser.add_argument("--checkpointing_limit", type=int, default=2)
    parser.add_argument("--resume_from_checkpoint", type=str, default="/home/zihao/CogVideo/finetune/finetune_output/checkpoint-70")
    
    # Validation Configuration
    parser.add_argument("--do_validation", type=bool, default=False)
    parser.add_argument("--validation_dir", type=str, default="/data/zihao/Disney-VideoGeneration-Dataset")
    parser.add_argument("--validation_steps", type=int, default=20)
    parser.add_argument("--validation_prompts", type=str, default="prompts.txt")
    parser.add_argument("--validation_images", type=str, default="images.txt")
    parser.add_argument("--validation_videos", type=str, default="")
    parser.add_argument("--gen_fps", type=int, default=16)
    
    parser.add_argument("--enable_slicing", type=bool, default=False)
    
    return parser.parse_args([]) 
args = get_training_args()
args.train_resolution = [int(x) for x in args.train_resolution.split('x')]
arg_dict = vars(args)
args = Args(**arg_dict)
args

Args(model_path=PosixPath('THUDM/CogVideoX-5B-I2V'), model_name='cogvideox-i2v', model_type='i2v', training_type='lora', output_dir=PosixPath('finetune_output'), report_to='tensorboard', tracker_name='finetrainer-cogvideo', data_root=PosixPath('/data/zihao/Disney-VideoGeneration-Dataset'), caption_column=PosixPath('prompt.txt'), image_column=PosixPath('images.txt'), video_column=PosixPath('videos.txt'), resume_from_checkpoint=PosixPath('/home/zihao/CogVideo/finetune/finetune_output/checkpoint-70'), seed=42, train_epochs=10, train_steps=None, checkpointing_steps=10, checkpointing_limit=2, batch_size=1, gradient_accumulation_steps=1, train_resolution=(49, 480, 720), mixed_precision='bf16', learning_rate=2e-05, optimizer='adamw', beta1=0.9, beta2=0.95, beta3=0.98, epsilon=1e-08, weight_decay=0.0001, max_grad_norm=1.0, lr_scheduler='constant_with_warmup', lr_warmup_steps=100, lr_num_cycles=1, lr_power=1.0, num_workers=8, pin_memory=True, gradient_checkpointing=True, enable_slicing=False, e

load model 

In [2]:
import sys
from pathlib import Path
from finetune.models.utils import get_model_cls
from finetune.schemas import Args
import torch
# os.environ["CUDA_VISIBLE_DEVICES"] = "7"
trainer_cls = get_model_cls(args.model_name, args.training_type)
trainer = trainer_cls(args)

/home/zihao/miniconda3/envs/cogvideo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zihao/miniconda3/envs/cogvideo/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Fetching 3 files: 100%|██████████| 3/3 [00:00<00:00, 22919.69it/s]
02/15/2025 10:24:50 - INFO - trainer - Initialized Trainer
02/15/2025 10:24:50 - INFO - trainer - Accelerator state: 
Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: bf16



In [ ]:
trainer.fit()

02/15/2025 10:24:50 - INFO - trainer - Initializing models
02/15/2025 10:24:50 - INFO - trainer - Initializing dataset and dataloader
02/15/2025 10:24:56 - INFO - trainer - Precomputing latent for video and prompt embedding ...
02/15/2025 10:25:00 - INFO - trainer - Precomputing latent for video and prompt embedding ... Done
02/15/2025 10:25:08 - INFO - trainer - Initializing trainable parameters


{'default': LoraConfig(task_type=None, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=128, target_modules={'to_v', 'to_out.0', 'to_k', 'to_q'}, exclude_modules=None, lora_alpha=64, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}


02/15/2025 10:25:12 - INFO - trainer - Initializing optimizer and lr scheduler
02/15/2025 10:25:32 - INFO - trainer - Initializing trackers
02/15/2025 10:25:32 - INFO - trainer - Starting training
02/15/2025 10:25:32 - INFO - trainer - Memory before training start: {
    "memory_allocated": 22.077,
    "memory_reserved": 22.088,
    "max_memory_allocated": 22.077,
    "max_memory_reserved": 22.088
}
02/15/2025 10:25:32 - INFO - trainer - Training configuration: {
    "trainable parameters": 132120576,
    "total samples": 69,
    "train epochs": 10,
    "train steps": 690,
    "batches per device": 1,
    "total batches observed per epoch": 69,
    "train batch size total count": 1,
    "gradient accumulation steps": 1
}
02/15/2025 10:25:32 - INFO - trainer - Resuming from checkpoint /home/zihao/CogVideo/finetune/finetune_output/checkpoint-70
02/15/2025 10:25:32 - INFO - accelerate.accelerator - Loading states from /home/zihao/CogVideo/finetune/finetune_output/checkpoint-70


[2025-02-15 10:25:32,836] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


02/15/2025 10:25:32 - INFO - root - gcc -pthread -B /home/zihao/miniconda3/envs/cogvideo/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/zihao/miniconda3/envs/cogvideo/include -fPIC -O2 -isystem /home/zihao/miniconda3/envs/cogvideo/include -fPIC -c /tmp/tmpmux1depa/test.c -o /tmp/tmpmux1depa/test.o
02/15/2025 10:25:32 - INFO - root - gcc -pthread -B /home/zihao/miniconda3/envs/cogvideo/compiler_compat /tmp/tmpmux1depa/test.o -laio -o /tmp/tmpmux1depa/a.out
/home/zihao/miniconda3/envs/cogvideo/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
02/15/2025 10:25:33 - INFO - root - gcc -pthread -B /home/zihao/miniconda3/envs/cogvideo/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/zihao/miniconda3/envs/cogvideo/include -fPIC -O2 -isystem /home/zihao/miniconda3/envs/cogvideo/include -fPIC -c /tmp/tmpma6lz2qz/test.c -o /tmp/tmpma6lz2qz/test.o
02/15/2025 10:25:33 - INFO - root - gcc -pthread -B /h

inference through finetuned model

In [6]:
import torch
from diffusers import AutoencoderKLCogVideoX, CogVideoXImageToVideoPipeline, CogVideoXTransformer3DModel
from diffusers.utils import export_to_video, load_image
from transformers import T5EncoderModel
import os
from peft import PeftModel
model_id = "THUDM/CogVideoX-5b-I2V"
torch.cuda.set_device(7)
device = "cuda:7"
lora_model_path = "finetune_output/checkpoint-690"

print(os.path.exists(lora_model_path))

transformer = CogVideoXTransformer3DModel.from_pretrained(model_id, subfolder="transformer", torch_dtype=torch.float16).to(device)

text_encoder = T5EncoderModel.from_pretrained(model_id, subfolder="text_encoder", torch_dtype=torch.float16).to(device)
vae = AutoencoderKLCogVideoX.from_pretrained(model_id, subfolder="vae", torch_dtype=torch.float16).to(device)

pipe = CogVideoXImageToVideoPipeline.from_pretrained(
    model_id,
    text_encoder=text_encoder,
    transformer=transformer,
    vae=vae,
    torch_dtype=torch.float16,
).to(device)

pipe.load_lora_weights(lora_model_path, weight_name="pytorch_lora_weights.safetensors", adapter_name="test_1")
pipe.fuse_lora(components=["transformer"], lora_scale=1 / 128)

pipe.enable_sequential_cpu_offload()

prompt = "An astronaut hatching from an egg, on the surface of the moon, the darkness and depth of space realised in the background. High quality, ultrarealistic detail and breath-taking movie-like camera shot."
image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/astronaut.jpg")

video = pipe(image=image, prompt=prompt, guidance_scale=6, use_dynamic_cfg=True, num_inference_steps=50).frames[0]

export_to_video(video, "output-finetuned.mp4", fps=8)

True


100%|██████████| 50/50 [06:05<00:00,  7.31s/it]


'output-finetuned.mp4'